In [2]:
import pandas as pd
import numpy as np

In [3]:
movie = pd.read_csv("..\data\processed\movies_clean.csv")
movies = movie.copy()

C:\Users\yasha\AppData\Local\Temp\ipykernel_22756\2665026983.py:1: DtypeWarning: Columns (7,12) have mixed types. Specify dtype option on import or set low_memory=False.
  movie = pd.read_csv("..\data\processed\movies_clean.csv")


In [4]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 318158 entries, 0 to 318157
Data columns (total 14 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    318158 non-null  int64  
 1   title                 318158 non-null  object 
 2   vote_average          318158 non-null  float64
 3   vote_count            318158 non-null  int64  
 4   release_date          306070 non-null  object 
 5   runtime               318158 non-null  int64  
 6   backdrop_path         318158 non-null  object 
 7   imdb_id               165872 non-null  object 
 8   popularity            318158 non-null  float64
 9   poster_path           318158 non-null  object 
 10  genres                260695 non-null  object 
 11  production_countries  244043 non-null  object 
 12  trailer_link          76371 non-null   object 
 13  combined_text         318158 non-null  object 
dtypes: float64(2), int64(3), object(9)
memory usage: 34.

In [5]:
# Convert all object/string columns to lowercase
for col in movies.select_dtypes(include='object').columns:
    movies[col] = movies[col].str.lower()


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [7]:
vector = TfidfVectorizer(stop_words='english', max_features=20000)
model = vector.fit_transform(movie['combined_text'])

In [8]:
from sklearn.decomposition import TruncatedSVD

In [9]:
svd = TruncatedSVD(n_components=200, random_state=42)
vector_reduced = svd.fit_transform(model)
vector_reduced = vector_reduced.astype('float32')


In [10]:
import joblib

In [11]:
with open("../models/vector.joblib", "wb") as f:
    joblib.dump(vector, f, protocol=4)

with open("../models/vector_reduced.joblib", "wb") as f:
    joblib.dump(vector_reduced, f, protocol=4)

with open("../models/svd_model.joblib", "wb") as f:
    joblib.dump(svd, f, protocol=4)

In [21]:
from huggingface_hub import hf_hub_download
vector = joblib.load(hf_hub_download("yashasvi-01-02/movie-recommendation", "models/vector.joblib"))
vector_reduced = joblib.load(hf_hub_download("yashasvi-01-02/movie-recommendation", "models/vector_reduced.joblib"))
svd = joblib.load(hf_hub_download("yashasvi-01-02/movie-recommendation", "models/svd_model.joblib"))
movie = pd.read_csv(hf_hub_download("yashasvi-01-02/movie-recommendation", "data/processed/movies_clean.csv"))


c:\Users\yasha\Desktop\movie_recommender\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
C:\Users\yasha\AppData\Local\Temp\

In [23]:
from sklearn.metrics.pairwise import cosine_similarity

def recommend_movies(query, top_n=10):

    movies_filtered = movie[(movie["vote_average"] > 2) & (movie["vote_count"] > 40)]
    query_vec = vector.transform([query.lower()])
    query_vec_reduced = svd.transform(query_vec)
    similarity = cosine_similarity(query_vec_reduced, vector_reduced[movies_filtered.index]).flatten()
    top_indices = similarity.argsort()[::-1][2:top_n+1]
    recommended = movies_filtered.iloc[top_indices][["id",'imdb_id','title','vote_average','vote_count','genres','backdrop_path','poster_path','trailer_link']].copy()
    recommended['similarity'] = similarity[top_indices]
    return recommended

query_result = recommend_movies("Comedy Romance hindi")
query_result.head(10)


,id,imdb_id,title,vote_average,vote_count,genres,backdrop_path,poster_path,trailer_link,similarity
30273,45002,tt0126234,Chamatkar,6.419,43,"Romance, Comedy",https://image.tmdb.org/t/p/original/fmSEpurLBh...,https://image.tmdb.org/t/p/original/tJlGNBkE0D...,NaN,0.800405
17427,505175,tt7853242,Love per Square Foot,6.693,106,"Comedy, Romance",https://image.tmdb.org/t/p/original/odmG3Y24tp...,https://image.tmdb.org/t/p/original/edKCIS7i4q...,https://www.youtube.com/watch?v=2I_eypETmKo,0.780098
12039,58128,tt0109117,Andaz Apna Apna,6.780,193,"Comedy, Romance, Family",https://image.tmdb.org/t/p/original/8YboYnoujB...,https://image.tmdb.org/t/p/original/a0GV2V3yif...,NaN,0.765876
19070,85052,tt1980986,Housefull 2,5.457,92,"Comedy, Romance",https://image.tmdb.org/t/p/original/fyAOfkkxou...,https://image.tmdb.org/t/p/original/y6L3GpfXWq...,https://www.youtube.com/watch?v=H40CULLKIyM,0.733410
17300,291157,tt3554418,Khoobsurat,6.700,107,"Comedy, Romance",https://image.tmdb.org/t/p/original/6OiopPJu3S...,https://image.tmdb.org/t/p/original/fJnWRipBGa...,https://www.youtube.com/watch?v=pOreDBy1YfI,0.731479
20541,19663,tt1275863,Love Aaj Kal,6.200,82,"Comedy, Drama, Romance",https://image.tmdb.org/t/p/original/vpCGx7IcYd...,https://image.tmdb.org/t/p/original/36hkylLil3...,https://www.youtube.com/watch?v=M8pWvwzbDQo,0.730259
26751,15467,tt1105733,Kismat Konnection,4.972,53,"Drama, Comedy, Romance",https://image.tmdb.org/t/p/original/pwSjIDnWw2...,https://image.tmdb.org/t/p/original/maIqwcXgvc...,NaN,0.721804
14208,103243,tt2317337,Vicky Donor,6.780,148,"Comedy, Romance",https://image.tmdb.org/t/p/original/gHDMBYpp5N...,https://image.tmdb.org/t/p/original/u8AO7DgcVY...,https://www.youtube.com/watch?v=jciL3-0Ix00,0.713537
29891,504080,tt7638344,Dhadak,7.295,44,"Romance, Drama, Comedy",https://image.tmdb.org/t/p/original/3BrGn1hzfT...,https://image.tmdb.org/t/p/original/vIVKzzPA7h...,https://www.youtube.com/watch?v=TIE92mUvSsw,0.705818
